# Kingspan API Client

This notebook contains a Python client for the Kingspan SOAP API based on the WSDL definition.

In [ ]:
# Install required dependencies
%pip install zeep requests


In [ ]:
from zeep import Client, Settings
from zeep.transports import Transport
from requests import Session
from datetime import datetime
from typing import Optional, List, Dict, Any
import os
from zeep.helpers import serialize_object


class KingspanAPIClient:
    """
    Python client for Kingspan SOAP API.
    
    This client provides methods to interact with Kingspan heating oil monitors
    through their SOAP web service API.
    """
    
    def __init__(self, wsdl_url: Optional[str] = None, timeout: int = 30):
        """
        Initialize the Kingspan API client.
        
        Args:
            wsdl_url: Path or URL to the WSDL file (defaults to kingspan_wsdl.xml in current directory)
            timeout: Request timeout in seconds
        """
        # Use default WSDL path if not provided
        if wsdl_url is None:
            # In notebook environment, look in the same directory as the notebook
            wsdl_url = os.path.join(os.getcwd(), "experimentation", "kingspan_wsdl.xml")
            # If that doesn't exist, try current working directory
            if not os.path.exists(wsdl_url):
                wsdl_url = os.path.join(os.getcwd(), "kingspan_wsdl.xml")
        
        # Verify file exists
        if not os.path.exists(wsdl_url):
            raise FileNotFoundError(
                f"WSDL file not found at: {wsdl_url}\n"
                f"Current working directory: {os.getcwd()}\n"
                f"Please ensure kingspan_wsdl.xml is in the experimentation directory."
            )
        
        print(f"Loading WSDL from: {wsdl_url}")
        
        # Read and validate WSDL file
        try:
            with open(wsdl_url, 'r', encoding='utf-8') as f:
                wsdl_content = f.read()
                
            # Check if file is empty or has issues
            if not wsdl_content.strip():
                raise ValueError("WSDL file is empty")
            
            # Check for common XML issues
            if not wsdl_content.strip().startswith('<?xml') and not wsdl_content.strip().startswith('<'):
                raise ValueError("WSDL file doesn't appear to be valid XML")
                
        except Exception as e:
            raise RuntimeError(f"Error reading WSDL file: {e}")
        
        session = Session()
        session.timeout = timeout
        transport = Transport(session=session)
        settings = Settings(strict=False, xml_huge_tree=True)
        
        try:
            self.client = Client(wsdl_url, transport=transport, settings=settings)
        except Exception as e:
            raise RuntimeError(
                f"Error parsing WSDL file at {wsdl_url}: {e}\n"
                f"The WSDL file may be corrupted or have invalid XML formatting."
            )
        
        self.email = None
        self.password = None
        self.user_id = None
        
    def authenticate_v3(self, email: str, password: str) -> Dict[str, Any]:
        """
        Authenticate with the Kingspan API (v3).
        
        Args:
            email: User's email address
            password: User's password
            
        Returns:
            Dictionary containing authentication result with user info and tanks
        """
        self.email = email
        self.password = password
        
        response = self.client.service.SoapMobileAPPAuthenicate_v3(
            emailaddress=email,
            password=password
        )
        result = self._parse_response(response)
        
        # Store the user ID for subsequent API calls
        self.user_id = result.get('APIUserID')
        
        return result
    
    def authenticate_v2(self, email: str, password: str) -> Dict[str, Any]:
        """Authenticate with the Kingspan API (v2)."""
        self.email = email
        self.password = password
        
        response = self.client.service.SoapMobileAPPAuthenicate_v2(
            emailaddress=email,
            password=password
        )
        return self._parse_response(response)
    
    def authenticate_v1(self, email: str, password: str) -> Dict[str, Any]:
        """Authenticate with the Kingspan API (v1)."""
        self.email = email
        self.password = password
        
        response = self.client.service.SoapMobileAPPAuthenicate_v1(
            emailaddress=email,
            password=password
        )
        return self._parse_response(response)
    
    def get_latest_level_v3(self, signalman_no: int, culture: str = "en-GB") -> Dict[str, Any]:
        """
        Get the latest tank level reading (v3).
        
        Args:
            signalman_no: Tank's signalman number
            culture: Culture code (default: en-GB)
            
        Returns:
            Dictionary containing latest level information and tank info
        """
        if not self.user_id or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPGetLatestLevel_v3(
            userid=self.user_id,
            password=self.password,
            signalmanno=signalman_no,
            culture=culture
        )
        return self._parse_response(response)
    
    def get_latest_level_v2(self, signalman_no: int, culture: str = "en-GB") -> Dict[str, Any]:
        """Get the latest tank level reading (v2)."""
        if not self.user_id or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPGetLatestLevel_v2(
            userid=self.user_id,
            password=self.password,
            signalmanno=signalman_no,
            culture=culture
        )
        return self._parse_response(response)
    
    def get_latest_level_v1(self, signalman_no: int, culture: str = "en-GB") -> Dict[str, Any]:
        """Get the latest tank level reading (v1)."""
        if not self.user_id or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPGetLatestLevel_v1(
            userid=self.user_id,
            password=self.password,
            signalmanno=signalman_no,
            culture=culture
        )
        return self._parse_response(response)
    
    def get_call_history_v1(
        self,
        signalman_no: int,
        start_date: datetime,
        end_date: datetime
    ) -> Dict[str, Any]:
        """
        Get historical tank level readings.
        
        Args:
            signalman_no: Tank's signalman number
            start_date: Start date for history
            end_date: End date for history
            
        Returns:
            Dictionary containing historical level readings
        """
        if not self.user_id or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPGetCallHistory_v1(
            userid=self.user_id,
            password=self.password,
            signalmanno=signalman_no,
            startdate=start_date,
            enddate=end_date
        )
        return self._parse_response(response)
    
    def get_user_account_details_v3(
        self,
        signalman_no: int,
        from_toolbox_app: bool = False
    ) -> Dict[str, Any]:
        """
        Get user account details (v3).
        
        Args:
            signalman_no: Tank's signalman number
            from_toolbox_app: Whether request is from toolbox app
            
        Returns:
            Dictionary containing user account and tank details
        """
        if not self.user_id or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPGetUserAccountDetails_v3(
            emailaddress=self.user_id,
            password=self.password,
            signalmanNo=signalman_no,
            fromToolBoxApp=from_toolbox_app
        )
        return self._parse_response(response)
    
    def get_user_account_details_v2(self, signalman_no: int) -> Dict[str, Any]:
        """Get user account details (v2)."""
        if not self.user_id or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPGetUserAccountDetails_v2(
            emailaddress=self.user_id,
            password=self.password,
            signalmanNo=signalman_no
        )
        return self._parse_response(response)
    
    def get_user_account_details_v1(self, signalman_no: int) -> Dict[str, Any]:
        """Get user account details (v1)."""
        if not self.user_id or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPGetUserAccountDetails_v1(
            emailaddress=self.user_id,
            password=self.password,
            signalmanNo=signalman_no
        )
        return self._parse_response(response)
    
    def check_device_status_v1(
        self,
        signalman_no: int,
        culture: str = "en-GB"
    ) -> Dict[str, Any]:
        """
        Check device status.
        
        Args:
            signalman_no: Tank's signalman number
            culture: Culture code
            
        Returns:
            Dictionary containing device status result
        """
        if not self.email or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPCheckDeviceStatus_v1(
            username=self.email,
            password=self.password,
            signalmanNo=signalman_no,
            culture=culture
        )
        return self._parse_response(response)
    
    def get_flo_dispense_events_v2(self, signalman_no: int) -> Dict[str, Any]:
        """
        Get flow dispense events (v2).
        
        Args:
            signalman_no: Tank's signalman number
            
        Returns:
            Dictionary containing dispense events and totals
        """
        if not self.user_id or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPGetFloDispenseEvents_v2(
            userid=self.user_id,
            password=self.password,
            signalmanno=signalman_no
        )
        return self._parse_response(response)
    
    def get_flo_dispense_events_v1(self, signalman_no: int) -> Dict[str, Any]:
        """Get flow dispense events (v1)."""
        if not self.user_id or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPGetFloDispenseEvents_v1(
            userid=self.user_id,
            password=self.password,
            signalmanno=signalman_no
        )
        return self._parse_response(response)
    
    def get_latest_smartserv_reading_v1(
        self,
        signalman_no: int,
        culture: str = "en-GB"
    ) -> Dict[str, Any]:
        """
        Get latest SmartServ reading.
        
        Args:
            signalman_no: Tank's signalman number
            culture: Culture code
            
        Returns:
            Dictionary containing SmartServ reading data
        """
        if not self.user_id or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPGetLatestSmartServReading_v1(
            userid=self.user_id,
            password=self.password,
            signalmanno=signalman_no,
            culture=culture
        )
        return self._parse_response(response)
    
    def get_languages_v1(self) -> List[Dict[str, str]]:
        """
        Get available languages.
        
        Returns:
            List of dictionaries containing language name/value pairs
        """
        if not self.email or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPGetLanguages_v1(
            emailaddress=self.email,
            password=self.password
        )
        return self._parse_response(response)
    
    def get_selected_language_v1(self, culture: str = "en-GB") -> List[Dict[str, str]]:
        """Get selected language details."""
        if not self.email or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPGetSelectedLanguage_v1(
            emailaddress=self.email,
            password=self.password,
            culture=culture
        )
        return self._parse_response(response)
    
    def get_tank_models_v1(self) -> List[Dict[str, str]]:
        """Get available tank models."""
        if not self.email or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPGetTankModels_v1(
            emailaddress=self.email,
            password=self.password
        )
        return self._parse_response(response)
    
    def get_strapping_table_by_tank_model_id_v1(
        self,
        tank_model_id: int
    ) -> Dict[str, Any]:
        """
        Get strapping table for a tank model.
        
        Args:
            tank_model_id: Tank model ID
            
        Returns:
            Dictionary containing strapping table data
        """
        if not self.email or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPGetStrappingTableByTankModelID_v1(
            emailaddress=self.email,
            password=self.password,
            tankModelID=tank_model_id
        )
        return self._parse_response(response)
    
    def get_device_type_v1(self, signalman_no: int, imei: int) -> int:
        """
        Get device type.
        
        Args:
            signalman_no: Tank's signalman number
            imei: Device IMEI
            
        Returns:
            Device type ID
        """
        if not self.email or not self.password:
            raise ValueError("Must authenticate first")
        
        response = self.client.service.SoapMobileAPPGetDeviceType_v1(
            emailaddress=self.email,
            password=self.password,
            signalmanNo=signalman_no,
            IMEI=imei
        )
        return response
    
    def hello_world(self) -> str:
        """Test method to verify API connectivity."""
        response = self.client.service.HelloWorld()
        return response
    
    @staticmethod
    def _parse_response(response) -> Any:
        """
        Parse SOAP response into Python dictionary.
        
        Args:
            response: SOAP response object
            
        Returns:
            Parsed response as dictionary or list
        """
        # Use zeep's built-in serializer for best results
        return serialize_object(response)


print("KingspanAPIClient class loaded successfully!")

## Usage Examples

## WSDL File Diagnostic

In [ ]:
# Initialize the client (WSDL file is automatically loaded from the current directory)
client = KingspanAPIClient()

# Test connectivity
try:
    result = client.hello_world()
    print(f"API Test: {result}")
except Exception as e:
    print(f"Connection test failed: {e}")

In [ ]:
# Example: Authenticate and get user info
email = "username"
password = "password"


In [ ]:
import pprint


try:
    # Authenticate using v3 API
    auth_result = client.authenticate_v3(email, password)
    
    print("Authentication Result:")
    print(f"User ID: {auth_result.get('APIUserID')}")
    print(f"Email: {auth_result.get('EmailAddress')}")
    print(f"Brand: {auth_result.get('ConsumerServicesBrandCode')}")
    
    # Display tank information
    tanks_raw = auth_result.get('Tanks')
    
    # Handle both dict with nested array and direct array
    if isinstance(tanks_raw, dict):
        # If Tanks is a dict, look for the array inside (e.g., APITankInfo_V3)
        tanks = tanks_raw.get('APITankInfo_V3') or tanks_raw.get('APITankInfo_V2') or tanks_raw.get('APITankInfo') or []
    elif isinstance(tanks_raw, list):
        tanks = tanks_raw
    else:
        tanks = []
    
    print(f"\nFound {len(tanks)} tank(s):")
    for tank in tanks:
        pprint.pp(tank)
        print(f"  - Tank: {tank.get('TankName')} (Signalman: {tank.get('SignalmanNo')})")
        print(f"    Level: {tank.get('LevelPercentage')}% ({tank.get('LevelLitres')} {tank.get('UnitOfMeasurement')})")
        
except Exception as e:
    import traceback
    print(f"Authentication failed: {e}")
    traceback.print_exc()

In [ ]:
# Example: Get latest tank level
signalman_no = 20026088  # Replace with your tank's signalman number

try:
    # Authenticate first
    auth_result = client.authenticate_v3(email, password)
    print(f"Authenticated as: {auth_result.get('EmailAddress')}")
    
    # Now get latest level
    level_data = client.get_latest_level_v3(signalman_no)
    
    # Debug: show raw response
    print("\nRaw API Response:")
    pprint.pp(level_data)
    
    # Check for API errors
    api_result = level_data.get('APIResult', {})
    if api_result.get('Code') != 0:
        print(f"\n❌ API Error: {api_result.get('Description')}")
    else:
        level = level_data.get('Level', {})
        print("\n✅ Latest Tank Reading:")
        print(f"  Reading Date: {level.get('ReadingDate')}")
        print(f"  Level: {level.get('LevelPercentage')}%")
        print(f"  Litres: {level.get('LevelLitres')}")
        print(f"  Consumption Rate: {level.get('ConsumptionRate')}")
        print(f"  Run Out Date: {level.get('RunOutDate')}")
        print(f"  Level Alert: {level.get('LevelAlert')}")
        print(f"  Drop Alert: {level.get('DropAlert')}")
    
except Exception as e:
    import traceback
    print(f"Failed to get level: {e}")
    traceback.print_exc()


In [ ]:
# Example: Get historical data
from datetime import datetime, timedelta

signalman_no = 20026088
end_date = datetime.now()
start_date = end_date - timedelta(days=30)  # Last 30 days

try:
    history = client.get_call_history_v1(signalman_no, start_date, end_date)
    
    # Handle nested structure - Levels is an OrderedDict with APILevel array inside
    levels_raw = history.get('Levels', {})
    if isinstance(levels_raw, dict):
        levels = levels_raw.get('APILevel') or []
    elif isinstance(levels_raw, list):
        levels = levels_raw
    else:
        levels = []
    
    print(f"Historical Data (last 30 days): {len(levels)} readings\n")
    
    for reading in levels[:10]:  # Show first 10 readings
        print(f"Date: {reading.get('ReadingDate')}")
        print(f"  Level: {reading.get('LevelPercentage')}% ({reading.get('LevelLitres')} litres)")
        consumption = reading.get('ConsumptionRate')
        if consumption and consumption != -1:
            print(f"  Consumption: {consumption} litres/day")
        print()
        
except Exception as e:
    import traceback
    print(f"Failed to get history: {e}")
    traceback.print_exc()


In [ ]:
# Example: Get user account details
import pprint

signalman_no = 20026088

try:
    account_details = client.get_user_account_details_v3(signalman_no)
    
    print("Raw Account Details Response:")
    pprint.pp(account_details)
    print("\n" + "="*80 + "\n")
    
    print("Account Details:")
    print(f"  User Name: {account_details.get('UserName')}")
    print(f"  Email: {account_details.get('UserEmail')}")
    print(f"  Tank Name: {account_details.get('TankName')}")
    print(f"  Tank Model: {account_details.get('TankModel')}")
    print(f"  Tank Capacity: {account_details.get('TankCapacity')} litres")
    print(f"  Fuel Type: {account_details.get('FuelType')}")
    print(f"  Tank Height: {account_details.get('TankHeightCM')} cm")
    
except Exception as e:
    import traceback
    print(f"Failed to get account details: {e}")
    traceback.print_exc()


## Helper Functions

In [ ]:
def get_all_tank_data(client: KingspanAPIClient, email: str, password: str) -> List[Dict]:
    """
    Helper function to get all tank data for a user.
    
    Args:
        client: KingspanAPIClient instance
        email: User email
        password: User password
        
    Returns:
        List of dictionaries with tank data
    """
    # Authenticate
    auth_result = client.authenticate_v3(email, password)
    
    tanks_data = []
    tanks = auth_result.get('Tanks', [])
    
    for tank in tanks:
        signalman_no = tank.get('SignalmanNo')
        
        try:
            # Get latest level
            level_data = client.get_latest_level_v3(signalman_no)
            
            tank_info = {
                'signalman_no': signalman_no,
                'name': tank.get('TankName'),
                'level_percentage': tank.get('LevelPercentage'),
                'level_litres': tank.get('LevelLitres'),
                'unit': tank.get('UnitOfMeasurement'),
                'latest_reading': level_data.get('Level', {}),
                'tank_info': level_data.get('TankInfo', [])
            }
            
            tanks_data.append(tank_info)
            
        except Exception as e:
            print(f"Error getting data for tank {signalman_no}: {e}")
    
    return tanks_data


print("Helper functions loaded!")

In [ ]:
# Check WSDL file
import os

wsdl_path = os.path.join(os.getcwd(), "experimentation", "kingspan_wsdl.xml")
print(f"Checking WSDL at: {wsdl_path}")
print(f"File exists: {os.path.exists(wsdl_path)}")

if os.path.exists(wsdl_path):
    with open(wsdl_path, 'r', encoding='utf-8') as f:
        content = f.read()
    print(f"File size: {len(content)} bytes")
    print(f"First 200 characters:\n{content[:200]}")
    
    # Check for problematic content
    if '<script' in content.lower():
        print("\n⚠️  WARNING: WSDL file contains <script> tags which may cause parsing issues")
        print("This might be an HTML file instead of a pure WSDL file")
else:
    print("\n❌ File not found!")

## Clean WSDL File

The WSDL file has HTML wrapper content that needs to be removed.

In [ ]:
import os
import re

wsdl_path = os.path.join(os.getcwd(), "experimentation", "kingspan_wsdl.xml")

# Read the current file
with open(wsdl_path, 'r', encoding='utf-8') as f:
    content = f.read()

print(f"Original file size: {len(content)} bytes")
print(f"First 100 chars: {content[:100]}")

# Remove script tags and their content
cleaned = re.sub(r'<script[^>]*>.*?</script>', '', content, flags=re.DOTALL | re.IGNORECASE)

# Find the start of actual XML (wsdl:definitions tag)
xml_start = cleaned.find('<wsdl:definitions')
if xml_start == -1:
    # Try without namespace prefix
    xml_start = cleaned.find('<definitions')

if xml_start != -1:
    # Extract just the XML part
    cleaned = cleaned[xml_start:]
    
    # Ensure it starts with XML declaration
    if not cleaned.startswith('<?xml'):
        cleaned = '<?xml version="1.0" encoding="utf-8"?>\n' + cleaned
    
    # Write the cleaned version
    with open(wsdl_path, 'w', encoding='utf-8') as f:
        f.write(cleaned)
    
    print(f"\n✅ WSDL file cleaned successfully!")
    print(f"New file size: {len(cleaned)} bytes")
    print(f"First 100 chars: {cleaned[:100]}")
else:
    print("\n❌ Could not find WSDL definitions in file")